# 2.2 - Linear Predictive Coding

<img style="float: right; margin:5px 0px 0px 10px" src="img/2-title.jpg" width="500">
Die lineare Vorhersagecodierung (LPC) ist ein Verfahren, das hauptsächlich in der Audiosignalverarbeitung und Sprachverarbeitung verwendet wird, um die Spektralhüllkurve eines digitalen Sprachsignals in komprimierter Form unter Verwendung der Informationen eines linearen Vorhersagemodells darzustellen. Es ist eine der leistungsstärksten Sprachanalysetechniken und eine der nützlichsten Methoden zum Codieren von Sprache guter Qualität mit einer niedrigen Bitrate und liefert hochgenaue Schätzungen von Sprachparametern. LPC ist die am weitesten verbreitete Methode in der Sprachcodierung und Sprachsynthese.

- Absender
    - Erstellen eines Beschreibungsmodells  
    - Extrahieren von charakteristischen Parametern, die sich auf das Sprachmodell aus dem Sprachsignal beziehen  

- Empfangsende
    - Bei der Sprachsynthese werden die entsprechenden Parameter durch entsprechende mathematische Modellberechnungen zur Sprachsynthese gesteuert

## Inhalt  
<table style="width:256px; border: 1px solid black; display: inline-block">
    <tr>
        <td  style="text-align:right" width=64px><img src="img/2-1.png" style="float:left"></td>
        <td style="text-align:left" width=256px>
            <a style="color:black; font-size:12px; font-weight:bold; text-decoration:none" href='#1'>
                1. Grundlagen
            </a>
        </td>
    </tr>  
    <tr>
        <td style="text-align:right"><img src="img/2-2.png" style="float:left"></td>
        <td style="text-align:left" width=128px>
            <a style="color:black; font-size:12px; font-weight:bold; text-decoration:none" href='#2'>
                2. Quelle-Filter
            </a>
        </td>
    </tr>
    <tr>
        <td style="text-align:right"><img src="img/2-3.jpg" style="float:left"></td>
        <td style="text-align:left" width=128px>
            <a style="color:black; font-size:12px; font-weight:bold; text-decoration:none" href='#3'>
                3. VocalTractLab
            </a>
        </td>
    </tr>
</table>

----

<a id='1'></a>
<div>
    <img src="img/2-1.png" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        1.  Grundlagen
    </h2>
</div>

LPC beginnt mit der Annahme, dass ein Sprachsignal von einem Summer am Ende einer Röhre erzeugt wird (stimmhafte Geräusche), mit gelegentlich hinzugefügten Zisch- und Knallgeräuschen (Zischlaute und plosive Geräusche). Obwohl dieses Modell anscheinend grob ist, ist es tatsächlich eine enge Annäherung an die Realität der Sprachproduktion. Die Glottis (der Raum zwischen den Stimmlippen) erzeugt das Summen, das durch seine Intensität (Lautstärke) und Frequenz (Tonhöhe) gekennzeichnet ist. Der Stimmapparat (Hals und Mund) bildet die Röhre, die durch ihre Resonanzen gekennzeichnet ist; Diese Resonanzen führen zu Formanten oder verbesserten Frequenzbändern im erzeugten Klang. Zischen und Knallen werden durch die Wirkung von Zunge, Lippen und Rachen während Zischlauten und Sprengstoffen erzeugt.

LPC analysiert das Sprachsignal, indem es die Formanten schätzt, ihre Auswirkungen aus dem Sprachsignal entfernt und die Intensität und Frequenz des verbleibenden Summens schätzt. Der Vorgang des Entfernens der Formanten wird als inverse Filterung bezeichnet, und das verbleibende Signal nach der Subtraktion des gefilterten modellierten Signals wird als Rest bezeichnet.

Die Zahlen, die die Intensität und Frequenz des Summens, die Formanten und das Rückstandssignal beschreiben, können gespeichert oder woanders übertragen werden. LPC synthetisiert das Sprachsignal durch Umkehren des Prozesses: Verwenden Sie die Buzz-Parameter und den Rest, um ein Quellensignal zu erstellen, verwenden Sie die Formanten, um einen Filter (der die Röhre darstellt) zu erstellen, und führen Sie die Quelle durch den Filter, was zu Sprache führt.

Da Sprachsignale mit der Zeit variieren, wird dieser Prozess an kurzen Abschnitten des Sprachsignals durchgeführt, die als Frames bezeichnet werden. Im Allgemeinen ergeben 30 bis 50 Bilder pro Sekunde eine verständliche Sprache mit guter Komprimierung.

----

<a id='2'></a>
<div>
    <img src="img/2-2.png" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        2. Quelle-Filter
    </h2>
</div>

Die Sprache wird von unserem Soundsystem erzeugt, das mit einer einfachen Schallquelle und einem Kanalmodell simuliert werden kann. Die Schallquelle wird von den Stimmbändern erzeugt. Die Stimmbänder liefern Erregungssignale für den Soundtrack. Diese Erregung kann periodisch oder nicht periodisch sein. Wenn sich die Stimmbänder in einem vokalisierten Zustand befinden (Vibration), werden Stimmgeräusche (zum Beispiel Vokale) erzeugt, wenn sich die Stimmbänder in einem stillen Zustand befinden, werden stille Geräusche (zum Beispiel Konsonanten) erzeugt. Der Kanal kann als Filter betrachtet werden, der das Spektrum des Anregungssignals von den Stimmbändern umformen kann, um verschiedene Klänge zu erzeugen. LPC ist die auf diesem Modell basierende Sprachgenerierungstechnologie. In diesem Modell wird das Sprachsignal durch ein Anregungssignal [Formel] durch ein zeitvariables Allpolfilter erzeugt. Der Koeffizient des Allpolfilters hängt von der Kanalform des spezifischen erzeugten Klangs ab. Das Anregungssignal [Formel] ist entweder eine Impulsfolge von stimmhafter Sprache oder zufälliges Rauschen von leisem Ton. Sprachsignal erzeugen [Formel] kann ausgedrückt werden als

----

<a id='3'></a>
<div>
    <img src="img/2-3.jpg" style="float:left">
    <h2 style="position: relative; top: 6px; left: 6px">
        3. VocalTractLab
    </h2>
</div>

VocalTractLab steht für "Vocal Tract Laboratory" und ist ein interaktives multimediales Softwaretool zur Demonstration des Mechanismus der Sprachproduktion. Es soll Studenten der Phonetik und verwandter Disziplinen ein intuitives Verständnis der Sprachproduktion ermöglichen.

VocalTractLab ist eine Open-Source-Software unter der GNU GPL (seit Version 2.3) und kostenlos. Weitere Informationen finden Sie unter [VocalTractLab](https://www.vocaltractlab.de/index.php?page=start).

----

### References

1. Titelbild von [Justin Stoltzfus](https://www.dataversity.net/voice-processing-are-we-near-new-speech-recognition-apps/#), [Emflazie](https://en.wikipedia.org/wiki/Source%E2%80%93filter_model#/media/File:Source-filter_model_diagram.svg), [adaptivedigital](https://www.adaptivedigital.com/lpc/)   
2. [Linear predictive coding](https://en.wikipedia.org/wiki/Linear_predictive_coding)
3. [Introduction - Linear predictive coding](http://support.ircam.fr/docs/AudioSculpt/3.0/co/LPC.html)  
4. [VocalTractLab](https://www.vocaltractlab.de/index.php?page=start)